# Chapter 13: The Average Causal Effect on the Treated Units and Other Estimands

In [1]:
from joblib import Parallel, delayed

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl

font = {'family' : 'IBM Plex Sans Condensed',
               'weight' : 'normal',
               'size'   : 10}
plt.rc('font', **font)
plt.rcParams['figure.figsize'] = (6, 6)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

np.random.seed(42)
%load_ext autoreload
%autoreload 1

%load_ext watermark
%watermark --iversions



seaborn          : 0.12.2
matplotlib_inline: 0.1.6
statsmodels      : 0.14.0
sklearn          : 1.3.2
numpy            : 1.24.3
matplotlib       : 3.7.2
pandas           : 2.0.3



In [2]:
def ATT_est(z, y, x, omod, pmod, ub=1):
    # E[Y | Z = 1]
    y0mean = y[z == 1].mean()
    nn, nn1 = len(z), z.sum()
    # fit pscore
    pscore = pmod.fit(x, z).predict_proba(x)[:, 1]
    pscore = np.clip(pscore, None, ub)
    odds = pscore / (1 - pscore)
    # fitted potential outcomes
    outcome0 = omod.fit(x[z == 0, :], y[z == 0]).predict(x)
    # omod
    ace_reg0 = sm.OLS(y, np.c_[sm.add_constant(z), x]).fit().params[1]
    ace_reg = y0mean - outcome0[z == 1].mean()
    # ipw
    ace_ipw0 = y0mean - (odds * (1 - z) * y).mean() * (nn / nn1)
    ace_ipw = y0mean - (odds * (1 - z) * y).mean() / (odds * (1 - z)).mean()
    # aipw
    res0 = y - outcome0
    ace_dr = ace_reg - (odds * (1 - z) * res0).mean() * (nn / nn1)
    return np.array([ace_reg0, ace_reg, ace_ipw0, ace_ipw, ace_dr])

In [3]:
from sklearn.linear_model import LogisticRegression, LinearRegression

lmod, omod = LogisticRegression(penalty=None), LinearRegression()

In [4]:
def OS_ATT(z, y, x, omod=omod, pmod=lmod, n_boot=2 * 1e2, Utruncps=1):
    n = len(z)
    point_est = ATT_est(z, y, x, omod, pmod, Utruncps)

    def bootfn(*args):
        # draw indices
        ids = np.random.choice(np.arange(n), size=n, replace=True)
        return ATT_est(z[ids], y[ids], x[ids, :], omod, pmod)

    boot_est = Parallel(n_jobs=-1)(delayed(bootfn)(i) for i in range(int(n_boot)))
    boot_est = np.vstack(boot_est)
    boot_se = boot_est.std(axis=0)
    res = pd.DataFrame(
        [point_est, boot_se],
        index=["point_est", "boot_se"],
        columns=["omod0", "omod", "ipw0", "ipw", "aipw"],
    )
    return res

## application

In [5]:
from sklearn.preprocessing import MinMaxScaler

nhanes_bmi = pd.read_csv("nhanes_bmi.csv").iloc[:, 1:]
nhanes_bmi.head()

z, y = (
    nhanes_bmi.School_meal.values,
    nhanes_bmi.BMI.values,
)
x = MinMaxScaler().fit_transform(X=nhanes_bmi.iloc[:, 2:].values)

In [6]:
(causaleffects := OS_ATT(z, y, x, n_boot=1e3, Utruncps=1))

,omod0,omod,ipw0,ipw,aipw
point_est,0.061248,-0.350718,-1.992439,-0.350810,-0.187104
boot_se,0.218705,0.244770,0.705875,0.320366,0.272267


In [7]:
(causaleffects := OS_ATT(z, y, x, n_boot=1e3, Utruncps=0.9))

,omod0,omod,ipw0,ipw,aipw
point_est,0.061248,-0.350718,-0.597019,-0.192312,-0.229505
boot_se,0.223913,0.252921,0.711488,0.336343,0.276487


with more flexible nuisance functions

In [14]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

rfc, rfr = (
    GradientBoostingClassifier(max_depth=3, random_state=0),
    GradientBoostingRegressor(max_depth=3, random_state=0),
)

In [16]:
(causaleffects := OS_ATT(z, y, x, omod=rfr, pmod=rfc, n_boot=1e3, Utruncps=0.9))

,omod0,omod,ipw0,ipw,aipw
point_est,0.061248,-0.171171,5.234295,-0.197837,-0.212641
boot_se,0.230824,0.287914,0.509390,0.312949,0.302220


## bonus: balancing weights

[calibration](https://github.com/google/empirical_calibration) package

In [8]:
import empirical_calibration as ec
from sklearn.preprocessing import PolynomialFeatures

Polynomial basis: linear and quadratic terms.

In [9]:
X = PolynomialFeatures(degree=2).fit_transform(x)

In [10]:
entr_weights, success = ec.calibrate(
    covariates=X[z == 0, :],
    target_covariates=X[z == 1, :],
    objective=ec.Objective.ENTROPY,
)
y[z == 1].mean() - np.sum(y[z == 0] * entr_weights)

-0.5635223070320698

In [11]:
l2_weights, success = ec.calibrate(
    covariates=X[z == 0, :],
    target_covariates=X[z == 1, :],
    objective=ec.Objective.QUADRATIC,
)
y[z == 1].mean() - np.sum(y[z == 0] * l2_weights)

-0.6197354773671719